In [14]:
BUCKET = 'gs://ecomm-query-product-pairs'

In [2]:
from datasets import load_dataset

raw_data = load_dataset("tasksource/esci")

In [21]:
import pandas as pd

LIMIT = 3000

for i, row in enumerate(raw_data["train"]):
    if i == LIMIT - 1:
        break
    elif i == 0:
        query_prod_pairs = pd.DataFrame({"content": [row["query"]], "type": ["query"]})
    else:
        query_prod_pairs = pd.concat(
            [query_prod_pairs,
            pd.DataFrame({"content": [row["query"]], "type": ["query"]})]
        )
    query_prod_pairs = pd.concat(
        [query_prod_pairs,
        pd.DataFrame({"content": [row["product_title"]], "type": ["product_title"]})]
    )

In [22]:
query_prod_pairs = query_prod_pairs.reset_index()
query_prod_pairs.head()

,index,content,type
0,0,revent 80 cfm,query
1,0,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,product_title
2,0,bathroom fan without light,query
3,0,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,product_title
4,0,revent 80 cfm,query


In [ ]:
! gsutil mb $BUCKET

In [25]:
output_file = 'batch_prediction_inputs.jsonl'

with open(output_file, 'w') as f:
    f.write(query_prod_pairs[['content']].to_json(lines=True, orient='records'))

In [30]:
! gsutil cp $output_file $BUCKET

Copying file://batch_prediction_inputs.jsonl [Content-Type=application/octet-stream]...
/ [1 files][506.0 KiB/506.0 KiB]                                                
Operation completed over 1 objects/506.0 KiB.                                    


In [37]:
! gsutil ls $BUCKET

gs://ecomm-query-product-pairs/batch_prediction_input.jsonl
gs://ecomm-query-product-pairs/batch_prediction_inputs.jsonl


In [35]:
from datetime import datetime
now = datetime.now()
now_string_tag = now.strftime("%Y_%m_%d_%H_%M_%S")

In [38]:
from vertexai.preview.language_models import TextEmbeddingModel
textembedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")
batch_prediction_job = textembedding_model.batch_predict(
  dataset=[f"{BUCKET}/{output_file}"],
  destination_uri_prefix=f"{BUCKET}/batch_predict_{now_string_tag}",
)
print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

Creating BatchPredictionJob


BatchPredictionJob created. Resource name: projects/679926387543/locations/us-central1/batchPredictionJobs/2589000169995370496
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/679926387543/locations/us-central1/batchPredictionJobs/2589000169995370496')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/2589000169995370496?project=679926387543
BatchPredictionJob projects/679926387543/locations/us-central1/batchPredictionJobs/2589000169995370496 current state:
JobState.JOB_STATE_QUEUED
BatchPredictionJob projects/679926387543/locations/us-central1/batchPredictionJobs/2589000169995370496 current state:
JobState.JOB_STATE_QUEUED
BatchPredictionJob projects/679926387543/locations/us-central1/batchPredictionJobs/2589000169995370496 current state:
JobState.JOB_STATE_QUEUED
